In [1]:
import numpy as np

In [2]:
def evalf(x):
  assert type(x) is np.ndarray 
  assert len(x)==2
  return 400*((x[0])**2)+0.004*((x[1])**4)

In [3]:
def evalh(x):
  assert type(x) is np.ndarray
  assert len(x)==2
  return np.array([[800,0],[0,0.048*((x[1])**2)]])

In [4]:
def evalg(x):
  assert type(x) is np.ndarray
  assert len(x)==2
  return np.array([800*x[0],0.016*((x[1])**3)])

In [5]:
def d_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  if np.linalg.det(evalh(x)) == 0:
    raise ValueError('Inverse does not exists. Please check!')
  return np.linalg.inv(evalh(x))

In [6]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p = - gradf
  D_k = d_k(x)
  r=rho
  y=gamma
  while evalf(x + alpha*np.matmul(D_k,p)) > evalf(x) + y*alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha=alpha*r
  return alpha

In [7]:
BACKTRACKING_LINE_SEARCH = 1
CONSTANT_STEP_LENGTH = 2

In [8]:
def find_minimizer_gdscaling(start_x, tol, line_search_type,*args):
  assert type(start_x) is np.ndarray and len(start_x) == 2 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
  k = 0
  while (np.linalg.norm(g_x) > tol): 
    D_k = d_k(x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma)
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) 
    k += 1
    g_x = evalg(x) 
  return x, k
   

#3. [R] Consider ηk = 1, ∀k = 1, 2, . . . in Algorithm 1. With starting point x0 = (2, 2) and a stopping toleranceτ = 10−9, find the number of iterations taken by the Newton’s method. Compare the number of iterations with that taken by Newton’s method (with backtracking line search) in Algorithm 1. Note the minimizer and minimum objective function value in each case. Comment on your observations.



#Solution:

In [9]:
my_start_x = np.array([2.0,2.0])
alpha_start = 1.0
rho = 0.5
gamma = 0.5
my_tol= 1e-9
x_opt_bls, k = find_minimizer_gdscaling(my_start_x, my_tol,CONSTANT_STEP_LENGTH)
print('Minimizer of the function when using constant step length with scaling:',x_opt_bls,'\nThe number of iterations using constant step length computation with scaling:',k,'\nMinimum value of the function by using constant step length with scaling:',evalf(x_opt_bls))


Minimizer of the function when using constant step length with scaling: [0.         0.00304488] 
The number of iterations using constant step length computation with scaling: 16 
Minimum value of the function by using constant step length with scaling: 3.4382653805813626e-13


In [10]:
my_start_x = np.array([2.0,2.0])
alpha_start = 1.0
rho = 0.5
gamma = 0.5
my_tol= 1e-9

x_opt_bls, k = find_minimizer_gdscaling(my_start_x, my_tol,BACKTRACKING_LINE_SEARCH,1.0,0.5,0.5)
#print(x_opt_bls)

print('Minimizer of the function when using backtracking line search with scaling:',x_opt_bls,'\nThe number of iterations when using backtracking line search computation with scaling:',k,'\nMinimum value of the function when using backtracking line search with scaling:',evalf(x_opt_bls))


Minimizer of the function when using backtracking line search with scaling: [0.         0.00304488] 
The number of iterations when using backtracking line search computation with scaling: 16 
Minimum value of the function when using backtracking line search with scaling: 3.4382653805813626e-13


When Using Constant step lenght line search in Newton"s Method:

The value of Minimizer: $[0 , 0.00304488]$

Minimum function value: $3.4382653805813626e-13$

Number of iterations:$16$


When using Backtracking line search with scaling in Newton's Method:

The value of Minimizer:$[0, .00304488]$ 


Minimum function value: $3.4382653805813626e-13$

Number of Iterations: $16$


Both the method have equal number of iterations
Both the method have  the value of minimizer and the optimal value of the function is equal

#4. [R] Compare the number of iterations obtained for the two variants of Newton’s method in the previous question, with that of gradient descent algorithm (without scaling) with backtracking line search, gradient descent algorithm (with scaling using a diagonal matrix) with backtracking line search (implemented in previouslabs), with starting point (2, 2). For backtracking line search, use α0 = 1, ρ = 0.5, γ = 0.5. Also compare the minimizer and minimum objective function value in each case. Comment on your observations.

#Solution:

In [11]:
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p=rho
  y=gamma
  while evalf(x+alpha*(-gradf)) > evalf(x)-y*alpha*np.dot((gradf.T),gradf):
    alpha=p*alpha
  return alpha
  

In [12]:
def find_minimizer_gd(start_x, tol, line_search_type,*args):
  assert type(start_x) is np.ndarray and len(start_x) == 2 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)
    k = 0
  while (np.linalg.norm(g_x) > tol): 
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,g_x)) 
    k += 1 
    g_x = evalg(x) 
  return x ,k  


In [13]:
my_start_x = np.array([2.0,2.0])
alpha_start = 1.0
rho = 0.5
gamma = 0.5
my_tol= 1e-9
x_opt_bls, k= find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1.0, 0.5,0.5)
print('Minimizer of the function when using backtracking line search with scaling:',x_opt_bls,'\nThe number of iterations when using backtracking line search computation with scaling:',k,'\nMinimum value of the function when using backtracking line search with scaling:',evalf(x_opt_bls))

Minimizer of the function when using backtracking line search with scaling: [0.         0.00304488] 
The number of iterations when using backtracking line search computation with scaling: 16 
Minimum value of the function when using backtracking line search with scaling: 3.4382653805813626e-13


In [14]:
my_start_x = np.array([2.0,2.0])
alpha_start = 1.0
rho = 0.5
gamma = 0.5
my_tol= 1e-9
x_opt_bls, k = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1.0, 0.5,0.5)
print('Minimizer of the function when using backtracking line search with scaling:',x_opt_bls,'\nThe number of iterations when using backtracking line search computation with scaling:',k,'\nMinimum value of the function when using backtracking line search with scaling:',evalf(x_opt_bls))




Params for Backtracking LS: alpha start: 1.0 rho: 0.5  gamma: 0.5


KeyboardInterrupt: ignored


When Using Backtracking line search with scaling

The value of Minimizer: $[0 ,0.00304488]$

Minimum function value: $ 3.4382653805813626e-13 $

Number of  of Iterations: $16$

when using Constant step length line search in Newton's Method :

The value of Minimizer: $[0, 0.00304488]$

Minimum function value: $3.4382653805813626e-13 $

Number  of Iterations: $16$

backtracking without scaling is taking large  number of iterations to terminateand backtracking  with scaling the number of iterations is 16

In constant step length in newton's method the number  of iterations is 16


